In [1]:
# import requests
# from bs4 import BeautifulSoup
import os
# import gzip
import shutil
# import cartopy.crs as ccrs
# import cartopy.feature
# import cartopy
# from metpy.plots import USCOUNTIES
from datetime import datetime, timedelta
# from PIL import Image
# import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
from pysteps import io, nowcasts, rcparams
from pysteps.motion.lucaskanade import dense_lucaskanade
# from pysteps.postprocessing.ensemblestats import excprob
from pysteps.utils import conversion, dimension, transformation
# from pysteps.visualization import plot_precip_field
# import matplotlib.colors as colors
import re
# import pandas as pd
from pysteps.nowcasts import anvil, extrapolation, sprog

Pysteps configuration file found at: C:\Users\16126\pysteps\pystepsrc



## Custom import functions

In [2]:
def import_mrms_dbz(filename, extent=None, window_size=1, **kwargs):
    import pygrib 
    import pyproj
    from functools import partial
    from pysteps.utils import aggregate_fields
    from pysteps.io.importers import _get_grib_projection, _get_threshold_value
    """
    Importer for NSSL's Multi-Radar/Multi-Sensor System
    ([MRMS](https://www.nssl.noaa.gov/projects/mrms/)) radar reflectivity product
    (grib format).
    Reflectivity values are expressed in dBZ and need to be converted to a rain rate in
    mm/h. The dimensions of the data
    array are [latitude, longitude]. The first grid point (0,0) corresponds to
    the upper left corner of the domain, while (last i, last j) denote the
    lower right corner.
    Due to the large size of the dataset (3500 x 7000), a float32 type is used
    by default to reduce the memory footprint. However, be aware that when this
    array is passed to a pystep function, it may be converted to double
    precision, doubling the memory footprint.
    To change the precision of the data, use the ``dtype`` keyword.
    Also, by default, the original data is downscaled by 4
    (resulting in a ~4 km grid spacing).
    In case that the original grid spacing is needed, use ``window_size=1``.
    But be aware that a single composite in double precipitation will
    require 186 Mb of memory.
    Finally, if desired, the data can be extracted over a
    sub region of the full domain using the `extent` keyword.
    By default, the entire domain is returned.
    Notes
    -----
    We replace any reflectivity values less than -35 with np.nan.
    
    Parameters
    ----------
    filename: str
        Name of the file to import.
    extent: None or array-like
        Longitude and latitude range (in degrees) of the data to be retrieved.
        (min_lon, max_lon, min_lat, max_lat).
        By default (None), the entire domain is retrieved.
        The extent can be in any form that can be converted to a flat array
        of 4 elements array (e.g., lists or tuples).
    window_size: array_like or int
        Array containing down-sampling integer factor along each axis.
        If an integer value is given, the same block shape is used for all the
        image dimensions.
        Default: window_size=4.
    {extra_kwargs_doc}
    Returns
    -------
    reflectivity: 2D array, float32
        Reflectivity field in dBZ. The dimensions are [latitude, longitude].
        The first grid point (0,0) corresponds to the upper left corner of the
        domain, while (last i, last j) denote the lower right corner.
    quality: None
        Not implement.
    metadata: dict
        Associated metadata (pixel sizes, map projections, etc.).
    """

    del kwargs

    try:
        grib_file = pygrib.open(filename)
    except OSError:
        raise OSError(f"Error opening NCEP's MRMS file. " f"File Not Found: {filename}")

    if isinstance(window_size, int):
        window_size = (window_size, window_size)

    if extent is not None:
        extent = np.asarray(extent)
        if (extent.ndim != 1) or (extent.size != 4):
            raise ValueError(
                "The extent must be None or a flat array with 4 elements.\n"
                f"Received: extent.shape = {str(extent.shape)}"
            )

    # The MRMS grib file contain one message with the precipitation intensity
    grib_file.rewind()
    grib_msg = grib_file.read(1)[0]  # Read the only message

    # -------------------------
    # Read the grid information

    lr_lon = grib_msg["longitudeOfLastGridPointInDegrees"]
    lr_lat = grib_msg["latitudeOfLastGridPointInDegrees"]

    ul_lon = grib_msg["longitudeOfFirstGridPointInDegrees"]
    ul_lat = grib_msg["latitudeOfFirstGridPointInDegrees"]

    # Ni - Number of points along a latitude circle (west-east)
    # Nj - Number of points along a longitude meridian (south-north)
    # The lat/lon grid has a 0.01 degrees spacing.
    lats = np.linspace(ul_lat, lr_lat, grib_msg["Nj"])
    lons = np.linspace(ul_lon, lr_lon, grib_msg["Ni"])

    reflectivity = grib_msg.values

    # Filter out noise
    no_data_mask = reflectivity <= -999.0 #-100
    #reflectivity[reflectivity <= -35] = np.nan


    # Create a function with default arguments for aggregate_fields
    block_reduce = partial(aggregate_fields, method="mean", trim=True)

    if window_size != (1, 1):
        # Downscale data
        lats = block_reduce(lats, window_size[0])
        lons = block_reduce(lons, window_size[1])

        # Update the limits
        ul_lat, lr_lat = lats[0], lats[-1]  # Lat from North to south!
        ul_lon, lr_lon = lons[0], lons[-1]

        reflectivity[no_data_mask] = 0  # block_reduce does not handle nan values
        reflectivity = block_reduce(reflectivity, window_size, axis=(0, 1))

        # Consider that if a single invalid observation is located in the block,
        # then mark that value as invalid.
        no_data_mask = block_reduce(
            no_data_mask.astype("int"), window_size, axis=(0, 1)
        ).astype(bool)

    lons, lats = np.meshgrid(lons, lats)
    reflectivity[no_data_mask] = np.nan

    if extent is not None:
        # clip domain
        ul_lon, lr_lon = _check_coords_range(
            (extent[0], extent[1]), "longitude", (ul_lon, lr_lon)
        )

        lr_lat, ul_lat = _check_coords_range(
            (extent[2], extent[3]), "latitude", (ul_lat, lr_lat)
        )

        mask_lat = (lats >= lr_lat) & (lats <= ul_lat)
        mask_lon = (lons >= ul_lon) & (lons <= lr_lon)

        nlats = np.count_nonzero(mask_lat[:, 0])
        nlons = np.count_nonzero(mask_lon[0, :])

        reflectivity = reflectivity[mask_lon & mask_lat].reshape(nlats, nlons)

    proj_params = _get_grib_projection(grib_msg)
    pr = pyproj.Proj(proj_params)
    proj_def = " ".join([f"+{key}={value} " for key, value in proj_params.items()])

    xsize = grib_msg["iDirectionIncrementInDegrees"] * window_size[0]
    ysize = grib_msg["jDirectionIncrementInDegrees"] * window_size[1]

    x1, y1 = pr(ul_lon, lr_lat)
    x2, y2 = pr(lr_lon, ul_lat)

    metadata = dict(
        institution="NOAA National Severe Storms Laboratory",
        xpixelsize=xsize,
        ypixelsize=ysize,
        unit="dBZ",
        accutime=2.0,
        transform="dB",
        zerovalue=-999.0,
        projection=proj_def.strip(),
        yorigin="upper",
        threshold=_get_threshold_value(reflectivity),
        x1=x1 - xsize / 2,
        x2=x2 + xsize / 2,
        y1=y1 - ysize / 2,
        y2=y2 + ysize / 2,
        cartesian_unit="degrees",
        zr_a = 316.0,
        zr_b = 1.5
    )

    return reflectivity, metadata

In [3]:
def read_radar_timeseries(filepath,start_date_str,num_previous_files):
    from glob import glob
    """
    Read a time series of input files using the import_mrms_dbz function 
    and stack them into a 3d array of
    shape (num_timesteps, height, width).
    Parameters
    ----------
    filepath: string
        Folder path where radar files lives. 
    start_date_str: string
        A date string of the most recent radar file to open (i.e. %Y%m%d-%H%M%S)
    num_previous_files: int
        Number of previous radar files to ingest.
    Returns
    -------
    out: tuple
        A three-element tuple containing the read data and quality rasters and
        associated metadata. If an input file name is None, the corresponding
        precipitation and quality fields are filled with nan values. If all
        input file names are None or if the length of the file name list is
        zero, a three-element tuple containing None values is returned.
    """

    reflectivity = []
    timestamps = []
    
    all_files = glob(os.path.join(filepath,'*.grib2'))
    if len(all_files)==0:
        raise OSError(f"Error. Not able to find grib2 files in " f": {filepath}")


    files_sorted = sorted(all_files)
    datelist = [re.search(r"(\d{8}-\d{6})", i).group() for i in files_sorted]

    ind = np.where(np.array(datelist)==start_date_str)[0]
    if len(ind)==0:
        raise OSError(f"Error finding matching grib2 file. No match for date: " f"{start_date_str}")

    files_keep = files_sorted[ind[0]-num_previous_files:ind[0]+1]
    
    for i in files_keep:
        radar_, metadata = import_mrms_dbz(i)
        reflectivity.append(radar_)
        timestamps.append(datetime.strptime(re.search(r"(\d{8}-\d{6})", i).group(), "%Y%m%d-%H%M%S")) # Get datetime from filename
    

    # Idk what this next line does, but it's in the pysteps code: https://github.com/pySTEPS/pysteps/blob/master/pysteps/io/readers.py#L77
    reflectivity = np.concatenate([radar_[None, :, :] for radar_ in reflectivity])
    metadata["timestamps"] = np.array(timestamps)

    return reflectivity, metadata

## Set File Path, Nowcast Parameters 

In [4]:
# Set MRMS file path
filepath = r"C:\Users\16126\pysteps_data\mrms_reflectivity\2023\06\15"
# Set the number of MRMS files, forecast timesteps and precip threshold to use in nowcast
num_previous_files = 4
n_leadtimes = 7
precip_thr = -150
number_cores = 2 #set to number of cores to use for calculation

## Load in MRMS Files

In [5]:
# Get a list of all files in the input directory
files = os.listdir(filepath)
files_sorted = sorted(files, reverse=True)
print(files_sorted[0])

MRMS_ReflectivityAtLowestAltitude_00.50_20230615-205441.grib2


In [6]:
# Extract the most recent MRMS file name from list of files
filename = files_sorted[0]

# Extract year, month, day, hour, minute, and seconds
year = filename[40:44]
month = filename[44:46]
day = filename[46:48]
hour = filename[49:51]
minute = filename[51:53]
second = filename[53:55]

In [7]:
start_date_str = f"{year}{month}{day}-{hour}{minute}{second}"

# Change this to be DBZ, metadata = .... then convert DBZ to R
R, metadata = read_radar_timeseries(filepath, start_date_str, num_previous_files)
print(R.shape)
print(metadata)

(5, 3500, 7000)
{'institution': 'NOAA National Severe Storms Laboratory', 'xpixelsize': 0.01, 'ypixelsize': 0.01, 'unit': 'dBZ', 'accutime': 2.0, 'transform': 'dB', 'zerovalue': -999.0, 'projection': '+proj=longlat  +ellps=IAU76', 'yorigin': 'upper', 'threshold': -6.0, 'x1': -129.99999999999997, 'x2': -60.00000199999991, 'y1': 20.000001, 'y2': 55.00000000000001, 'cartesian_unit': 'degrees', 'zr_a': 316.0, 'zr_b': 1.5, 'timestamps': array([datetime.datetime(2023, 6, 15, 20, 22, 42),
       datetime.datetime(2023, 6, 15, 20, 30, 41),
       datetime.datetime(2023, 6, 15, 20, 38, 42),
       datetime.datetime(2023, 6, 15, 20, 46, 40),
       datetime.datetime(2023, 6, 15, 20, 54, 41)], dtype=object)}


## Converstions / Upscaling

In [8]:
# Convert dbZ to rain rate (similar to MCH example)
RR, metadata_RR = conversion.to_rainrate(R, metadata)
print(RR.shape)

# Upscale data to 4 km to limit memory usage
R_down, metadata_down = dimension.aggregate_fields_space(RR, metadata_RR, 0.04)
print(R_down.shape)

(5, 3500, 7000)
(5, 875, 1750)


In [9]:
# R_down, metadata_down
R = R_down
metadata = metadata_down

R = conversion.to_reflectivity(R,metadata)[0]

## <font color='Black'>Calculate Motion</font>

In [10]:
from pysteps import motion
DBR, metadata_DBR = transformation.dB_transform(R_down, metadata_down, threshold=0.1, zerovalue=-15)
V = dense_lucaskanade(DBR)

## <font color='Orange'>ANVIL</font>

In [11]:
import time
start_time = time.time()

forecast_anvil = anvil.forecast(
    R_down[-4:, :, :],    
    V,
    n_leadtimes,
    n_cascade_levels=8,
    # precip_thr=precip_thr,
    extrap_method='semilagrangian',
    # probmatching_method="cdf",
    ar_order = 2,
    ar_window_radius=25,
    num_workers = number_cores
)

end_time = time.time()
execution_time = end_time - start_time
# num_prev_files = num_previous_files
# method = 'ANVIL'
# df = pd.read_csv(r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\calc_duration.csv")
# # Create a new row with the variables as columns
# new_row = {"method": method,"num_prev_files": num_prev_files,"n_ens_members": n_ens_members,"n_leadtimes": n_leadtimes,"seed": seed,"execution_time": execution_time,}
# # Convert the new row to a DataFrame
# new_row_df = pd.DataFrame([new_row])
# # Concatenate the original DataFrame with the new row DataFrame
# df = pd.concat([df, new_row_df], ignore_index=True)
# # Save the modified DataFrame to a new CSV file
# df.to_csv(r"G:\My Drive\PowPow\Resort Graphs\forecast_radar\calc_duration.csv", index=False)

print(execution_time)

Computing ANVIL nowcast
-----------------------

Inputs
------
input dimensions: 875x1750

Methods
-------
extrapolation:   semilagrangian
FFT:             numpy

Parameters
----------
number of time steps:        7
parallel threads:            2
number of cascade levels:    8
order of the ARI(p,1) model: 2
ARI(p,1) window radius:      25
R(VIL) window radius:        3
Starting nowcast computation.
Computing nowcast for time step 1... done.
Computing nowcast for time step 2... done.
Computing nowcast for time step 3... done.
Computing nowcast for time step 4... done.
Computing nowcast for time step 5... done.
Computing nowcast for time step 6... done.
Computing nowcast for time step 7... done.
38.01028609275818


## Convert ANVIL to dBZ

In [12]:
forecast_anvil_dbz, metadata_forecast = conversion.to_reflectivity(forecast_anvil,metadata)

In [13]:
print(metadata_forecast['timestamps'])

cur_time = np.max(metadata['timestamps'])
print(cur_time)
new_timestamps = []
for i in range(forecast_anvil_dbz.shape[0]):
    new_timestamps.append(cur_time+timedelta(minutes=8))
    cur_time+=timedelta(minutes=8)
    
print(new_timestamps)

metadata_forecast['timestamps'] = new_timestamps

[datetime.datetime(2023, 6, 15, 20, 22, 42)
 datetime.datetime(2023, 6, 15, 20, 30, 41)
 datetime.datetime(2023, 6, 15, 20, 38, 42)
 datetime.datetime(2023, 6, 15, 20, 46, 40)
 datetime.datetime(2023, 6, 15, 20, 54, 41)]
2023-06-15 20:54:41
[datetime.datetime(2023, 6, 15, 21, 2, 41), datetime.datetime(2023, 6, 15, 21, 10, 41), datetime.datetime(2023, 6, 15, 21, 18, 41), datetime.datetime(2023, 6, 15, 21, 26, 41), datetime.datetime(2023, 6, 15, 21, 34, 41), datetime.datetime(2023, 6, 15, 21, 42, 41), datetime.datetime(2023, 6, 15, 21, 50, 41)]


## Export

In [14]:
# print(forecast_anvil_dbz.shape)
# print(metadata_forecast)

In [15]:
# print(forecast_anvil_dbz[i,:,:][np.newaxis,:,:].shape)

In [16]:
# from pysteps.io.exporters import initialize_forecast_exporter_netcdf,close_forecast_files,export_forecast_dataset

# metadata_forecast_single = metadata_forecast.copy()
# for i in range(forecast_anvil_dbz.shape[0]):
#     metadata_forecast_single['timestamps'] = [metadata_forecast['timestamps'][i]]
#     print(metadata_forecast_single['timestamps'])
    
#     tstamp = datetime.strftime(metadata_forecast['timestamps'][i],'%Y%m%d%H%M')

#     exporter = initialize_forecast_exporter_netcdf(r'.','mrms_nowcast_dbz5_'+tstamp, metadata_forecast['timestamps'][i], 8, 
#                                                    1, (875, 1750), metadata_forecast_single, 1,'member')

#     export_forecast_dataset(forecast_anvil_dbz[i,:,:][np.newaxis,:,:],exporter)
#     close_forecast_files(exporter)

In [17]:
metadata_forecast

{'institution': 'NOAA National Severe Storms Laboratory',
 'xpixelsize': 0.04,
 'ypixelsize': 0.04,
 'unit': 'dBZ',
 'accutime': 2.0,
 'transform': 'dB',
 'zerovalue': -10.999999999999996,
 'projection': '+proj=longlat  +ellps=IAU76',
 'yorigin': 'upper',
 'threshold': -5.999999999999997,
 'x1': -129.99999999999997,
 'x2': -60.00000199999991,
 'y1': 20.000001,
 'y2': 55.00000000000001,
 'cartesian_unit': 'degrees',
 'zr_a': 316.0,
 'zr_b': 1.5,
 'timestamps': [datetime.datetime(2023, 6, 15, 21, 2, 41),
  datetime.datetime(2023, 6, 15, 21, 10, 41),
  datetime.datetime(2023, 6, 15, 21, 18, 41),
  datetime.datetime(2023, 6, 15, 21, 26, 41),
  datetime.datetime(2023, 6, 15, 21, 34, 41),
  datetime.datetime(2023, 6, 15, 21, 42, 41),
  datetime.datetime(2023, 6, 15, 21, 50, 41)]}

In [18]:
# from pysteps.io.exporters import initialize_forecast_exporter_netcdf,close_forecast_files,export_forecast_dataset

# # initialize_forecast_exporter_netcdf(outpath =r'.', outfnprefix ='rain_rate', date,timestep = 8, n_timesteps = 7, shape=(875, 1750), metadata=metadata, n_ens_members=28,incremental='timestep')
# exporter = initialize_forecast_exporter_netcdf(r'.','mrms_nowcast_dbz', date,8, 7, (875, 1750), metadata_forecast, 28,'member')
# print(exporter)

# export_forecast_dataset(forecast_anvil_dbz,exporter)
# close_forecast_files(exporter)